# Loading Data 

Para o experimentos foram escolhidos XXX movimentos que foram coletados da seguinte maneira:

Para cada postura de mão, o "subject" realizou cada movimento durante 5s por vez, com um intervalo de outros 5s para repouso, para um total de 10 repetições de cada gesto. Dessa forma, obtém-se 10 amostras de cada gesto de mão, cada uma durando 5 segundos, ou seja, 5000 pontos coletados. 

O repouso foi coletado de entre alguns dos movimentos.

Os movimentos são então separados ein:

0. gesture_0: hand resting | gesto 0: mão em repouso:
1. gesture_1: close hand   | gesto 1: mão fechada

In [ ]:
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import hmmlearn

gesture_samples = 2
n_gestures = 2
file_name = 'https://raw.githubusercontent.com/raphaelrcb/TCC_UnB/raphael_dev/data/gesture_0_1.csv'

# rest_hand_series = read_csv(file_name, header=1, usecols = [0,1], index_col = 0)
rest_hand_series = []
close_hand_series = []

for i in range(gesture_samples):
    rest_hand_file =    file_name[:len(file_name)-5] + str(i+1) + file_name[len(file_name)-4:]
    close_hand_file =   file_name[:len(file_name)-7] + "1_" + str(i+1) + file_name[len(file_name)-4:]
    print(rest_hand_file)
    rest_hand_series.append(read_csv(rest_hand_file, header=1, usecols = [0,1], index_col = 0))
    rest_hand_series[i].index = pd.to_datetime(rest_hand_series[i].index, unit = 's', origin= 'unix')
    rest_hand_series[i].index = rest_hand_series[i].index.map(lambda t: t.replace(year=2021, month=10, day=31, hour=00))
    rest_hand_series[i].index = pd.DatetimeIndex(rest_hand_series[i].index).to_period('L')
    print(close_hand_file)
    close_hand_series.append(read_csv(close_hand_file, header=1, usecols = [0,1], index_col = 0))
    close_hand_series[i].index = pd.to_datetime(close_hand_series[i].index, unit = 's', origin= 'unix')
    close_hand_series[i].index = close_hand_series[i].index.map(lambda t: t.replace(year=2021, month=10, day=31, hour=00))
    close_hand_series[i].index = pd.DatetimeIndex(close_hand_series[i].index).to_period('L')

print(rest_hand_series[0])

# Segmenting Data

Como os dados para cada movimento foram coletados em uma única "rodada", ou seja, em uma coleta contínua para as 10 observações do gesto, é necessária uma segmentação manual para se separar apenas os 5s necessários de cada observação. Isso é feito offline através do software do delsys.

Como cada amostra de movimento dura 5s e é coletado numa freqûencia de 1000Hz, tem-se um total de 5000 pontos de dados, que podem ser divididos em janelas de 150ms com sobreposição de 50ms. Dessa forma, seguindo a fórmula

$$
N\; janelas = \frac{tamanho\: dos\,dados - tamanho\, da\, janela}{incremento\, da\,janela} + 1
$$

É possível obter um total de 98 janelas por segmento de dado.

In [ ]:
data_size = len(close_hand_series[0])
window_size = 150
window_overlap = 50

n_windows = (data_size - window_size)/window_overlap + 1

print(n_windows)

open_hand_window = []
close_hand_window = []

rest_hand_window = []
close_hand_window = []

for j in range (gesture_samples):
    for i in range(int(n_windows)):
        rest_hand_window.append( rest_hand_series[j][(i*window_overlap):(i*window_overlap + window_size)] )
        close_hand_window.append( close_hand_series[j][(i*window_overlap):(i*window_overlap + window_size)] )
        # osw.append( series[(i*window_overlap):(i*window_overlap + window_size)])

rest_hand_window[0]
# close_hand_window[0]

# Feature Extraction

Dado que para cada gesto, obteve-se XX janelas de dados, serão extraídas os coeficientes AR de quarta ordem de cada janela. Metade das janelas serão usadas para treinamento enquanto a outra metade será usada para testar o modelo.  

Se são 98 janelas por gesto, e cada gesto repetido 2 vezes, então tem-se 196 janelas. Define-se que 98 delas serão usadas para treinamento do HMM e as outras 98 para teste. Para o treinamento offline, os coeficientes AR de quarta ordem serão extraídos de cada janela, gerando 196 (98 x 2) vetores de características que possuem 4 dimensões no total.


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

rest_features = []
close_features = []

for i in range(int(n_windows)):
    #extracting for rest gesture
    rest_model = ARIMA(rest_hand_window[i], order = (4,0,0))
    rest_fit   = rest_model.fit(method='yule_walker')
    rest_features.append(rest_fit.polynomial_ar)
    rest_features[i] = np.delete(rest_features[i], 0)
    rest_features[i] = -rest_features[i]
    # #extracting for close gesture
    close_model = ARIMA(close_hand_window[i], order = (4,0,0))
    close_fit   = close_model.fit(method='yule_walker')
    close_features.append(close_fit.polynomial_ar)
    close_features[i] = np.delete(close_features[i], 0)
    close_features[i] = -close_features[i]

# close_features
# print(rest_fit.summary())
# print(rest_features[0])
# print(rest_features[5])
# print(rest_features[30])
# print(rest_features[60])
print(rest_features[70])

´´´
rest hand                                                       close hand
        0           5           30          60      90                  0              5        30      60      90      
L1 = 1.1050     =  0.9055   =  0.8249 =  0.9839 =  1.1135   | L1 =  1.4683      =  1.2648 =  1.3483 =  1.2895 =  1.4038 
L2 = -0.3798    = -0.3409   = -0.1025 = -0.4266 = -0.5778   | L2 = -1.0909      = -0.8506 = -1.0147 = -0.9046 = -1.1744
L3 = -0.0817    =  0.0115   =  0.0251 = -0.0081 =  0.1528   | L3 =  0.4278      =  0.1225 =  0.2858 =  0.2004 =  0.5062
L4 = 0.0126     = -0.1474   = -0.2659 = -0.0320 = -0.1106   | L4 = -0.2065      = -0.0804 = -0.0889 = -0.0473 = -0.1544


# Contrução do Modelo HMM